<a href="https://colab.research.google.com/github/jakubglinka/google.colab/blob/master/word2vec_nkjp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CBOW word2vec model on National Korpus of Polish Language

## Outline:
 
 - prepare data
 - train embeddings
 - train sentiment classifier

Remarks: use Tensorflow 2.0:

https://github.com/tensorflow/docs/blob/master/site/en/r2/guide/effective_tf2.md

### TODO's:
  - nkjp class with following functionalities:
    - download
    - summary
    - split into train, valid and test (using whole texts I guess) stratified by type of text
    - generate random sample of n sentences (segment on the fly) from traning/test/validation part
    - train segmentor
    - prepare tokenisation
  - sentencepeice tokenisation

### Install dependencies:

In [55]:
!pip install untangle tf-nightly-2.0-preview sentencepiece nltk wget tqdm

# Prepare corpus

http://nkjp.pl/settings/papers/NKJP_ksiazka.pdf

*   Segment tokenizer using NLTK
*   Training folder (for both sentence pieces and word2vec embedding)
*   Test folder



In [57]:
!rm -rf nkjp* NKJP*
!ls

m.model  m.vocab  sample_data  text.txt


In [59]:
import subprocess
import tqdm

class tqdm_wget_pbar:
  def __enter__(self):
    self.elapsed = None
    self.tqdm = None
    return self

  def update(self, elapsed, total, done):
    if self.tqdm is None:
      self.tqdm = tqdm.tqdm(total=round(total / 1e6), unit="MB", unit_scale=True)
      self.tqdm.update(round(elapsed / 1e6, 2))
      self.elapsed = elapsed
    else:
      self.tqdm.update(round((elapsed - self.elapsed) / 1e6, 5))
      self.elapsed = elapsed

  def __exit__(self, b, c, d):
    self.tqdm.close()

class NKJP:
 
  def __init__(self, dir: str, url: str = None):
    self.url = "http://clip.ipipan.waw.pl/NationalCorpusOfPolish?action=AttachFile&do=get&target=NKJP-PodkorpusMilionowy-1.2.tar.gz"
    self.dir = dir
  
  def is_downloaded(self) -> bool:  
    return os.path.exists(self.dir)
  
  def download(self):
        
    if self.is_downloaded() == False:
      
      os.mkdir(self.dir)
      with tqdm_wget_pbar() as pbar:
        _pbar = lambda elapsed, total, done: pbar.update(elapsed, total, done) 
        wget.download(url=self.url, 
                      out= self.dir + "/nkjp.tar.gz",
                      bar=_pbar)      
    else:
      print("nkjp already downloaded, reusing...")

  
nkjp = NKJP("./nkjp/")
print(nkjp.is_downloaded())
nkjp.download()

!ls


True
nkjp already downloaded, reusing...
m.model  m.vocab  nkjp	sample_data  text.txt


# Text pre-processing:

http://www.aclweb.org/anthology/P16-1162

  - sentencepiece

In [2]:
!echo "To jest mój tekst w języku Polskim ." > text.txt
!echo "Zdanie drugie ." >> text.txt

!head text.txt

To jest mój tekst w języku Polskim .
Zdanie drugie .


In [0]:
!pwd

/content


In [5]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train("--input=/content/text.txt --model_prefix=m --vocab_size=13 --model_type=word")
sp = spm.SentencePieceProcessor()
sp.Load("/content/m.model")


sp.EncodeAsPieces("To jest mój tekst w języku Polskim , słowo nieznane .")
ids = sp.EncodeAsIds("""To jest mój tekst w języku Polskim , słowo nieznane . To jest drugie zdanie .""")
print(ids)
sp.DecodeIds(ids)

# sp.EncodeAsPieces("This is a test")

[5, 8, 10, 11, 12, 9, 4, 0, 3, 5, 8, 7, 0, 3]


'To jest mój tekst w języku Polskim ⁇  . To jest drugie ⁇  .'

In [0]:
spm

True

In [0]:
import tensorflow as tf
import untangle as unt

In [0]:
# download and extract korpus from ipipan website

!wget "http://clip.ipipan.waw.pl/NationalCorpusOfPolish?action=AttachFile&do=get&target=NKJP-PodkorpusMilionowy-1.2.tar.gz" -O nkjp.tar.gz
!mkdir nkjp
!tar -C ./nkjp -zxf nkjp.tar.gz -o
!ls

In [0]:
!ls ./nkjp/ | head -n 2
!cat ./nkjp/030-2-000000001/text.xml

010-2-000000001
030-2-000000001
<?xml version="1.0" encoding="UTF-8"?>
<teiCorpus xmlns:xi="http://www.w3.org/2001/XInclude" xmlns="http://www.tei-c.org/ns/1.0">
 <xi:include href="NKJP_1M_header.xml"/>
 <TEI>
  <xi:include href="header.xml"/>
  <text xml:id="txt_text" xml:lang="pl">
   <body xml:id="txt_body">
    <div xml:id="txt_1-div" decls="#h_1-bibl">
     <ab n="p882in890of:PWN:030-2-000000001" xml:id="txt_1.1-ab">Łzy padały na cremoński lakier. Szkoda, nie wolno było niszczyć przedmiotu, na który ojciec, biedaczysko, wydał całą schedę po Luizie... Otarła je lewą, umęczoną ręką.</ab>
     <ab n="p883in891of:PWN:030-2-000000001" xml:id="txt_1.2-ab">Wszedł Adam. Zawiało wodą kwiatową Maréchal Niel. Starannie domykał drzwi za sobą.</ab>
     <ab n="p884in892of:PWN:030-2-000000001" xml:id="txt_1.3-ab">Trwała dalej w bezruchu.</ab>
    </div>
    <div xml:id="txt_2-div" decls="#h_2-bibl">
     <ab n="p45in53of:PWN:030-2-000000001" xml:id="txt_2.1-ab">Pan dyrektor był nieobecny, a Róż

In [0]:
xx = unt.parse("./nkjp/030-2-000000001/text.xml")
[xxx.cdata for xxx in xx.teiCorpus.TEI.text.body.div[11].ab]

['Róża opuściła ręce. Siadła - nogi drżały. Rzuciła smyczek...',
 'Księżycowa orkiestra pod batutą Brahmsa grała dalej. Tylko na miejscu skrzypiec wystąpiła cisza - czarna jak zaskórna woda. Jeszcze tu i ówdzie błyskał refleks sola... cisza przecież czyniła się coraz głębsza, szersza i pochłaniała resztkę wibracji. Z wolna milkły także widmowe instrumenty. Po jednemu wsiąkały w próżnię... Chaos bezdźwięczny pokrył wreszcie harmonię.']

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.0.0-dev20190217'

In [0]:
x = tf.random.normal([2, 2])

@tf.function
def square(x: tf.Tensor) -> tf.Tensor:
  return(tf.square(x))

print(x)
print(square(x))

W0217 10:25:40.323168 140638079080320 tf_logging.py:161] Entity <function square at 0x7fe8b0f67f28> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY=5. Please report this to the AutoGraph team. Cause: Unexpected error transforming <function square at 0x7fe8b0f67f28>. If you believe this is due to a bug, please set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output when filing the bug report. Caused by: name 'tf' is not defined


tf.Tensor(
[[-0.15044403  1.6586291 ]
 [ 1.1326104  -1.2539116 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.02263341 2.7510505 ]
 [1.2828064  1.5722944 ]], shape=(2, 2), dtype=float32)


In [0]:
??tf.function